In [ ]:
function gs_compatible(gsd::String, gsm::String)
    """
    Teste la compatibilité du groupe sanguin d'un donneur avec celui d'un malade 

    Entrées :
        - gsd : String - Groupe sanguin du donneur
        - gsm : String - Groupe sanguin d'un malade
    
    Sortie :
        - Boolean : Compatibilité sanguine calculée
    """
    if (gsd == "O") || (gsm == "AB")
        # O peut donner à {O,A,B,AB}, AB peut recevoir de {O,A,B,AB}
        return true
    elseif (gsd == "A") && (gsm == "A")
        #  A donne à {A,AB}
        return true
    elseif (gsd == "B") && (gsm == "B")
        # B donne à {B,AB}
        return true
    else
        # O ne peut pas recevoir de {A,B,AB}, A ne peut pas recevoir {B,AB}, B ne peut pas recevoir de {A,AB}
        return false;
    end
end

In [ ]:
function read_dat_file(dat_file::String)
    """
    Construit le graphe des compatibilités "a priori" (G_tilde) à partir du fichier `.dat` de la PrefLib.

    Entrée :
        - dat_file : String - Chemin du fichier `.dat`.
    
    Sortie :
        - Gtilde : MetaDiGraph - Graphe G_tilde du fichier
    """
    # Message d'erreur si le fichier n'existe pas
    isfile(dat_file) || throw(ArgumentError(".dat file not found."))

    # Extraction des données individuelles depuis le fichier .dat
    file = readdlm(dat_file, '\n')
    nb_vertices = length(file)-1
    Gtilde = MetaDiGraph(nb_vertices, 0)
    
    # Lecture des lignes du fichier
    for line in file[2:end]
        splitted_line = split(line, ',')
        pair = parse(Int, splitted_line[1])
        set_prop!(Gtilde, pair, :gsm, String(splitted_line[2])) # groupe sanguin du malade
        set_prop!(Gtilde, pair, :gsd, String(splitted_line[3])) # groupe sanguin du donneur
        set_prop!(Gtilde, pair, :pra, parse(Float64, splitted_line[5])) # PRA du malade
    end

    # Construction du graphe de compatibilité a priori ̃G
    # --> on fixe le poids des arcs à 1 par défaut 
    
    for u in 1:nb_vertices
        for v in u+1:nb_vertices
            if gs_compatible(get_prop(Gtilde, u, :gsd), get_prop(Gtilde,v, :gsm))
                add_edge!(Gtilde, u, v, :weight, 1.0)
            end
            if gs_compatible(get_prop(Gtilde, v, :gsd), get_prop(Gtilde,u, :gsm))
                add_edge!(Gtilde, v, u, :weight, 1.0)
            end
        end
    end

    return Gtilde
end

In [ ]:
function get_failure_rates(kep_graph::MetaDiGraph, distribution::String)
    """
    Calcule les probabilité d'échec de chaque arc du graphe, et remplace le poids de l'arc par cette valeur. 

    Entrées :
        - kep_graph    : MetaDiGraph - Graphe décrivant les paires et leurs compatibilités
        - distribution : String      - Type de distribution pour les incertitudes (parmi les distributions du vecteur DISTRIBUTIONS)
    """
    
    #failure_rates = []

    for edge in edges(kep_graph)
        # La probabilité d'échec dépend de la distribution choisir
        if distribution == "Constant"
            # Probabilité d'échec constant, fixé à 0.7
            set_prop!(kep_graph, edge, :failure, 0.7)
        
        elseif distribution == "Binomial"
            if rand() < 0.25
                # Probabilité moyenne de 0.1 pour 25% des arcs 
                set_prop!(kep_graph, edge, :failure, rand() * 0.2)
            else
                # Probabilité moyenne de 0.9 pour 75% des arcs
                set_prop!(kep_graph, edge, :failure, 0.8 + rand() * 0.2)
            end
            
        elseif distribution == "BinomialUNOS"
            # %pra correspond au "panel reactive antibody level"
            # Si le %pra est inférieur à 0.8 : Le patient est "UNOS low sensitized" 
            if get_prop(kep_graph, edge.dst, :pra) < 0.8
                # Probabilité d'échec de 0.1 pour de tels patients 
                set_prop!(kep_graph, edge, :failure, 0.1)
            else
                # Probabilité d'échec de 0.9 dans le cas contraire
                set_prop!(kep_graph, edge, :failure, 0.9)
            end
            
        elseif distribution  == "BinomialAPD"
            # Si le %pra est inférieur à 0.75 : Le patient est "APD low sensitized"
            if get_prop(kep_graph, edge.dst, :pra) < 0.75
                # Probabilité d'échec de 0.28 pour de tels patients 
                set_prop!(kep_graph, edge, :failure, 0.28)
            else
                # Probabilité d'échec de 0.58 dans le cas contraire
                set_prop!(kep_graph, edge, :failure, 0.58)
            end
            
        elseif distribution == "NoFailure"
            # Cadre idéal : les probabilités d'échecs valent toutes 0
            set_prop!(kep_graph, edge, :failure, 0.)
        end
    end
end


In [ ]:
println("Fonctions créant les graphes importées avec succès.")